# Interactive Brokers Connection Example

This notebook demonstrates basic connection and operations with Interactive Brokers using the ib_async library.

## Connection

**Important Note**: From now on, use `ib_async` instead of the older Interactive Brokers API.

In [4]:
from ib_async import *
util.startLoop()  # required in Jupyter (interactive environments), not in scripts

In [5]:
ib = IB()

**Make sure you are logged in your TWS!!**

In [6]:
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [8]:
ib.positions()

[]

In [9]:
ib.disconnect()

'Disconnecting from 127.0.0.1:7497, 117 B sent in 8 messages, 18.9 kB received in 398 messages, session time 5.85 s.'

## Contracts (Introduction)

Working with contracts is fundamental in Interactive Brokers. A contract represents a financial instrument and contains all the information needed to uniquely identify it on the exchange.

Here we'll create and qualify a Forex contract. The `qualifyContracts()` function fills in missing contract details by querying Interactive Brokers' database.

In [10]:
from ib_async import *
util.startLoop()

In [11]:
ib = IB()

In [12]:
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [13]:
contract = Forex("EURUSD")
contract

Forex('EURUSD', exchange='IDEALPRO')

Create a Forex contract for EUR/USD. The `Forex()` constructor creates a contract object, but it's not yet validated with IB:

In [14]:
contract = ib.qualifyContracts(contract)
contract

[Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')]

The `qualifyContracts()` function validates the contract with IB and fills in missing details like contract ID, exchange info, etc.:

In [15]:
type(contract)

list

Check the type - `qualifyContracts()` returns a list, even for a single contract:

In [16]:
contract[0]

Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')

Access the first (and only) contract in the returned list to see the full contract details:

In [17]:
contract[0].conId

12087792

The `conId` (contract ID) is a unique identifier assigned by Interactive Brokers to each financial instrument:

## Contract Types

Interactive Brokers supports various contract types. Each type has specific parameters:

- **Contract(conId)**: Direct reference using IB's contract ID
- **Stock**: Equity securities (symbol, exchange, currency, primaryExchange optional)
- **Forex**: Currency pairs (currency pair like 'EURUSD')
- **CFD**: Contracts for Difference (symbol)
- **Future**: Futures contracts (symbol, expiry date, exchange)
- **Option**: Options contracts (symbol, expiry, strike, right, exchange)
- **Bond**: Fixed income securities (using ISIN or other identifiers)

In [18]:
# Contract(conId=270639)
# Stock('AMD', 'SMART', 'USD')
# Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
# Forex('EURUSD')
# CFD('IBUS30')
# Future('ES', '20180921', 'GLOBEX')
# Option('SPY', '20170721', 240, 'C', 'SMART')
# Bond(secIdType='ISIN', secId='US03076KAA60')

### Handling Ambiguous vs Unambiguous Contracts

When creating contracts, some may be **ambiguous** (multiple matches) while others are **unambiguous** (single match):

- **Unambiguous**: When you provide enough details (symbol + exchange + currency), IB can uniquely identify the contract
- **Ambiguous**: When details are incomplete (e.g., just symbol), IB may find multiple matching contracts

The `qualifyContracts()` function behaves differently in each case:

In [19]:
contract = Stock("AAPL", "SMART", "USD")
contract

Stock(symbol='AAPL', exchange='SMART', currency='USD')

**Unambiguous case**: Full contract details provided (symbol, exchange, currency):

In [20]:
contract = ib.qualifyContracts(contract)  # unambiguous
contract

[Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS')]

This should return a single qualified contract since the details uniquely identify AAPL:

In [21]:
contract = ib.qualifyContracts(Stock("AAPL"))  # ambiguous
contract

Ambiguous contract: Stock(symbol='AAPL'), possibles are [Contract(secType='STK', conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), Contract(secType='STK', conId=273982664, symbol='AAPL', exchange='SMART', primaryExchange='EBS', currency='CHF', localSymbol='AAPL', tradingClass='AAPL'), Contract(secType='STK', conId=532640894, symbol='AAPL', exchange='SMART', primaryExchange='TSE', currency='CAD', localSymbol='AAPL', tradingClass='AAPL'), Contract(secType='STK', conId=265598, symbol='AAPL', exchange='AMEX', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), Contract(secType='STK', conId=265598, symbol='AAPL', exchange='NYSE', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), Contract(secType='STK', conId=265598, symbol='AAPL', exchange='CBOE', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), Contract(secType='ST

[None]

**Ambiguous case**: Only symbol provided, no exchange or currency specified. This may return multiple contracts if AAPL trades on different exchanges or in different currencies:

In [22]:
ib.disconnect()

'Disconnecting from 127.0.0.1:7497, 250 B sent in 11 messages, 49.6 kB received in 434 messages, session time 36.8 s.'

Finally, disconnect from Interactive Brokers when done:

# Current Market Data

This section demonstrates how to retrieve real-time market data for financial instruments using Interactive Brokers.

In [23]:
import pandas as pd
from ib_async import *
util.startLoop()

In [24]:
ib = IB()

In [26]:
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [27]:
contract = Forex("EURUSD")
contract = ib.qualifyContracts(contract)[0]  # Qualify and get first result
contract

Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')

In [28]:
data1 = ib.reqMktData(contract)
data1

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), defaults=IBDefaults(emptyPrice=-1, emptySize=0, unset=nan, timezone=datetime.timezone.utc), created=True)

In [29]:
data1

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2025, 11, 6, 20, 7, 45, 424804, tzinfo=datetime.timezone.utc), timestamp=1762459665.4248161, minTick=1e-05, bid=1.15462, bidSize=7500000.0, ask=1.15464, askSize=1500000.0, last=1.15465, lastSize=0.0, lastTimestamp=datetime.datetime(2025, 11, 6, 20, 6, 50, tzinfo=datetime.timezone.utc), prevBid=1.15464, prevBidSize=5000000.0, prevAsk=1.15465, prevAskSize=9000000.0, volume=0.0, high=1.15525, low=1.1491, close=1.1493, ticks=[TickData(time=datetime.datetime(2025, 11, 6, 20, 7, 45, 424804, tzinfo=datetime.timezone.utc), tickType=1, price=1.15462, size=7500000.0), TickData(time=datetime.datetime(2025, 11, 6, 20, 7, 45, 424804, tzinfo=datetime.timezone.utc), tickType=2, price=1.15464, size=1500000.0)], defaults=IBDefaults(emptyPrice=-1, emptySize=0, unset=nan, timezone=datetime.timezone.utc), created=True)

In [30]:
data1.ask

1.15464

In [31]:
data1.askSize

1500000.0

In [32]:
data1.marketPrice()

1.15463

In [33]:
data1.time

datetime.datetime(2025, 11, 6, 20, 7, 52, 478927, tzinfo=datetime.timezone.utc)

In [34]:
data1.time

datetime.datetime(2025, 11, 6, 20, 7, 54, 380514, tzinfo=datetime.timezone.utc)

In [35]:
pd.to_datetime(data1.time)

Timestamp('2025-11-06 20:07:55.543921+0000', tz='UTC')

In [36]:
contract = Stock("AAPL", "SMART", "USD")
contract

Stock(symbol='AAPL', exchange='SMART', currency='USD')

In [37]:
contract = Forex("EURUSD")
contract = ib.qualifyContracts(contract)[0]  # Qualify and get first result
contract
data2 = ib.reqMktData(contract)
data2

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2025, 11, 6, 20, 7, 59, 818951, tzinfo=datetime.timezone.utc), timestamp=1762459679.818957, minTick=1e-05, bid=1.15462, bidSize=10000000.0, ask=1.15464, askSize=5000000.0, last=1.15465, lastSize=0.0, lastTimestamp=datetime.datetime(2025, 11, 6, 20, 6, 50, tzinfo=datetime.timezone.utc), prevBid=1.15464, prevBidSize=9000000.0, prevAsk=1.15465, prevAskSize=5500000.0, volume=0.0, high=1.15525, low=1.1491, close=1.1493, defaults=IBDefaults(emptyPrice=-1, emptySize=0, unset=nan, timezone=datetime.timezone.utc), created=True)

In [40]:
ib.disconnect()

# Data Streaming for Multiple Tickers

This section demonstrates how to stream real-time market data for multiple financial instruments simultaneously. This is useful for monitoring portfolios, arbitrage opportunities, or building real-time dashboards.

In [46]:
import pandas as pd
from ib_async import *
import time
util.startLoop()

In [47]:
ib = IB()
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

## Setup Multiple Contracts

First, let's define multiple contracts we want to stream data for:

In [48]:
# Define contracts for different asset types
contracts = [
    Forex("EURUSD"),    # EUR/USD forex pair
    Forex("GBPUSD"),    # GBP/USD forex pair  
    Forex("USDJPY"),    # USD/JPY forex pair
]

# Qualify all contracts
qualified_contracts = []
for contract in contracts:
    qualified = ib.qualifyContracts(contract)
    if qualified:
        qualified_contracts.append(qualified[0])

print(f"Successfully qualified {len(qualified_contracts)} contracts:")
for contract in qualified_contracts:
    print(f"  - {contract.symbol}: {contract}")

Successfully qualified 3 contracts:
  - EUR: Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')
  - GBP: Forex('GBPUSD', conId=12087797, exchange='IDEALPRO', localSymbol='GBP.USD', tradingClass='GBP.USD')
  - USD: Forex('USDJPY', conId=15016059, exchange='IDEALPRO', localSymbol='USD.JPY', tradingClass='USD.JPY')


## Start Market Data Streams

Now let's request market data for all contracts and store the ticker objects:

In [49]:
# Request market data for all contracts
tickers = []
for contract in qualified_contracts:
    ticker = ib.reqMktData(contract)
    tickers.append(ticker)

print(f"Started streaming data for {len(tickers)} instruments")

# Wait a moment for data to start flowing
ib.sleep(2)

# Display initial data
for i, ticker in enumerate(tickers):
    print(f"{ticker.contract.symbol}: Bid={ticker.bid}, Ask={ticker.ask}, Last={ticker.last}")

Started streaming data for 3 instruments
EUR: Bid=1.15484, Ask=1.15485, Last=1.1548
GBP: Bid=1.31394, Ask=1.31395, Last=1.31395
USD: Bid=153.093, Ask=153.096, Last=153.095


## Real-Time Data Streaming Loop

Now let's create a loop that continuously displays updated market data for all our instruments. This demonstrates how you might monitor multiple assets in real-time:

In [51]:
# Debug: Check what we actually have
print("Available tickers:")
for ticker in tickers:
    print(f"  - Symbol: {ticker.contract.symbol}, Contract: {ticker.contract}")

# Create mapping for easier access
ticker_map = {ticker.contract.symbol: ticker for ticker in tickers}
print(f"\nTicker map keys: {list(ticker_map.keys())}")

# Check if we have the expected symbols
required_symbols = ['EURUSD', 'GBPUSD', 'USDJPY']
available_symbols = list(ticker_map.keys())

if not all(symbol in available_symbols for symbol in required_symbols):
    print(f"Warning: Not all required symbols available.")
    print(f"Required: {required_symbols}")
    print(f"Available: {available_symbols}")
    
    # Use whatever symbols we have
    if len(available_symbols) >= 3:
        data1 = tickers[0]  # First available
        data2 = tickers[1]  # Second available  
        data3 = tickers[2]  # Third available
        print(f"Using: {data1.contract.symbol}, {data2.contract.symbol}, {data3.contract.symbol}")
    else:
        print("Not enough tickers available. Please check connection and contracts.")
        # Exit early if not enough data
else:
    # Get references to individual tickers for the loop
    data1 = ticker_map['EURUSD']  # EUR/USD
    data2 = ticker_map['GBPUSD']  # GBP/USD  
    data3 = ticker_map['USDJPY']  # USD/JPY

if len(tickers) >= 3:
    print("Starting real-time data stream...")
    print("Symbol       | Market Price 1 | Market Price 2 | Market Price 3")
    print("-" * 60)

    # Stream data for 10 iterations (similar to the screenshot example)
    for i in range(10):
        price1 = round(data1.marketPrice(), 5) if data1.marketPrice() else 'N/A'
        price2 = round(data2.marketPrice(), 3) if data2.marketPrice() else 'N/A'
        price3 = round(data3.marketPrice(), 3) if data3.marketPrice() else 'N/A'
        print(f"Update {i+1:2d}    | {price1:>13} | {price2:>13} | {price3:>13}")
        ib.sleep(1)
else:
    print("Cannot start streaming - insufficient ticker data")

Available tickers:
  - Symbol: EUR, Contract: Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')
  - Symbol: GBP, Contract: Forex('GBPUSD', conId=12087797, exchange='IDEALPRO', localSymbol='GBP.USD', tradingClass='GBP.USD')
  - Symbol: USD, Contract: Forex('USDJPY', conId=15016059, exchange='IDEALPRO', localSymbol='USD.JPY', tradingClass='USD.JPY')

Ticker map keys: ['EUR', 'GBP', 'USD']
Required: ['EURUSD', 'GBPUSD', 'USDJPY']
Available: ['EUR', 'GBP', 'USD']
Using: EUR, GBP, USD
Starting real-time data stream...
Symbol       | Market Price 1 | Market Price 2 | Market Price 3
------------------------------------------------------------
Update  1    |       1.15474 |         1.314 |       153.104
Update  2    |       1.15474 |         1.314 |       153.105
Update  3    |       1.15474 |         1.314 |       153.105
Update  4    |       1.15474 |         1.314 |       153.103
Update  5    |       1.15474 |         1.314 |       153.105
U

## Alternative: Simple Loop Format 

A simple loop that prints market prices:

In [52]:
for i in range(10):
    print(round(data1.marketPrice(), 5), round(data2.marketPrice(), 3), round(data3.marketPrice(), 3))
    ib.sleep(1)

1.15471 1.314 153.106
1.15472 1.314 153.106
1.15472 1.314 153.108
1.15475 1.314 153.106
1.15475 1.314 153.107
1.15475 1.314 153.107


KeyboardInterrupt: 

## Creating a Data Streaming DataFrame

For more advanced analysis, you might want to collect the streaming data into a pandas DataFrame:

In [53]:
# Collect data into a list for DataFrame creation
data_records = []

print("Collecting data for DataFrame...")
for i in range(5):
    timestamp = pd.Timestamp.now()
    record = {
        'timestamp': timestamp,
        'EURUSD': data1.marketPrice(),
        'GBPUSD': data2.marketPrice(), 
        'USDJPY': data3.marketPrice()
    }
    data_records.append(record)
    print(f"Sample {i+1}: {record}")
    ib.sleep(1)

# Create DataFrame
df = pd.DataFrame(data_records)
df.set_index('timestamp', inplace=True)
print("\nCollected data as DataFrame:")
print(df)

Sample 1: {'timestamp': Timestamp('2025-11-06 20:21:47.360686'), 'EURUSD': 1.1547450000000001, 'GBPUSD': 1.313715, 'USDJPY': 153.108}
Sample 2: {'timestamp': Timestamp('2025-11-06 20:21:48.363140'), 'EURUSD': 1.1547450000000001, 'GBPUSD': 1.313715, 'USDJPY': 153.108}
Sample 3: {'timestamp': Timestamp('2025-11-06 20:21:49.364693'), 'EURUSD': 1.1547450000000001, 'GBPUSD': 1.313715, 'USDJPY': 153.108}
Sample 4: {'timestamp': Timestamp('2025-11-06 20:21:50.367164'), 'EURUSD': 1.1547450000000001, 'GBPUSD': 1.313715, 'USDJPY': 153.10750000000002}
Sample 5: {'timestamp': Timestamp('2025-11-06 20:21:51.371008'), 'EURUSD': 1.1547450000000001, 'GBPUSD': 1.313715, 'USDJPY': 153.10750000000002}

Collected data as DataFrame:
                              EURUSD    GBPUSD    USDJPY
timestamp                                               
2025-11-06 20:21:47.360686  1.154745  1.313715  153.1080
2025-11-06 20:21:48.363140  1.154745  1.313715  153.1080
2025-11-06 20:21:49.364693  1.154745  1.313715  15

## Clean Up - Cancel Market Data

When you're done streaming, it's important to cancel the market data subscriptions to free up resources:

In [54]:
# Cancel market data subscriptions
for ticker in tickers:
    ib.cancelMktData(ticker.contract)
    print(f"Cancelled market data for {ticker.contract.symbol}")

print("All market data subscriptions cancelled.")

# Disconnect from IB
ib.disconnect()

Cancelled market data for EUR
Cancelled market data for GBP
Cancelled market data for USD
All market data subscriptions cancelled.


'Disconnecting from 127.0.0.1:7497, 516 B sent in 17 messages, 53.7 kB received in 1799 messages, session time 148 s.'

# Troubleshooting: Client ID Already in Use

If you get "Error 326: client id is already in use", here are the solutions:

In [55]:
# Solution 1: Use a different client ID
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=2)  # Use clientId=2 instead of default 1

<IB connected to 127.0.0.1:7497 clientId=2>

In [56]:
# Solution 2: Force disconnect any existing connections first
try:
    ib.disconnect()
except:
    pass  # Ignore if already disconnected

# Wait a moment and reconnect
import time
time.sleep(2)
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [57]:
# Solution 3: Restart TWS/IB Gateway
# If the above doesn't work, you need to:
# 1. Close TWS (Trader Workstation) or IB Gateway completely
# 2. Restart the application
# 3. Then reconnect with clientId=1

# Solution 4: Use a helper function for robust connection
def connect_ib(max_client_id=10):
    """Try to connect with different client IDs until one works"""
    for client_id in range(1, max_client_id + 1):
        try:
            ib = IB()
            ib.connect('127.0.0.1', 7497, clientId=client_id)
            print(f"Connected successfully with clientId={client_id}")
            return ib
        except Exception as e:
            print(f"Failed to connect with clientId={client_id}: {e}")
            continue
    
    raise Exception(f"Could not connect with any clientId from 1 to {max_client_id}")

# Usage:
# ib = connect_ib()